In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

In [12]:
raw_data_df = pd.read_csv('..//Deep-Learning-project/resources/charity_data.csv')
raw_data_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [13]:
alphabet_df = raw_data_df.drop(['EIN', 'NAME'], axis=1)
alphabet_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [14]:
print(alphabet_df.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [15]:
alphabet_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [16]:
cut_off = 500

application_types_to_replace = alphabet_df['APPLICATION_TYPE'].value_counts()[alphabet_df['APPLICATION_TYPE'].value_counts() < cut_off].index.tolist()

for app in application_types_to_replace:
    alphabet_df['APPLICATION_TYPE'] = alphabet_df['APPLICATION_TYPE'].replace(app, 'Other')
    
alphabet_df['APPLICATION_TYPE'].value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [17]:
alphabet_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1248        1
C2561        1
C1820        1
C1728        1
C3700        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [18]:
alphabet_df['CLASSIFICATION'].value_counts()[alphabet_df['CLASSIFICATION'].value_counts() > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C4100        6
C1720        6
C2400        6
C1257        5
C1600        5
C2710        3
C0           3
C1260        3
C1234        2
C3200        2
C1267        2
C1256        2
C1246        2
Name: CLASSIFICATION, dtype: int64

In [19]:
cut_off = 1500

classifications_to_replace = alphabet_df['CLASSIFICATION'].value_counts()[alphabet_df['CLASSIFICATION'].value_counts() < cut_off].index.tolist()

for cls in classifications_to_replace:
    alphabet_df['CLASSIFICATION'] = alphabet_df['CLASSIFICATION'].replace(cls, 'Other')


alphabet_df['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [20]:
alphabet_df['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [21]:
alphabet_df = alphabet_df[alphabet_df['STATUS'] == 1]
alphabet_df.shape

(34294, 10)

In [22]:
ask_amt = alphabet_df['ASK_AMT']

quartiles = np.quantile(ask_amt,[.25,.75])
iqr = quartiles[1]-quartiles[0]
lower_bound = quartiles[0]-(1.5*iqr)
upper_bound = quartiles[1]+(1.5*iqr)

potential_outliers = (ask_amt <= upper_bound) | (ask_amt >= lower_bound)
alphabet_df = alphabet_df.loc[potential_outliers,:]

alphabet_df.shape


(34294, 10)

In [23]:
app_dummies_df = pd.get_dummies(alphabet_df)
app_dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [24]:
y = app_dummies_df['IS_SUCCESSFUL'].values
X = app_dummies_df.drop(['IS_SUCCESSFUL'], axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [25]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [26]:

nn = tf.keras.models.Sequential()

nn_input_dim = X_train.shape[1]


nn.add(tf.keras.layers.Dense(units=10, activation='tanh', input_dim=nn_input_dim))


nn.add(tf.keras.layers.Dense(units=10, activation='tanh'))


nn.add(tf.keras.layers.Dense(units=10, activation='tanh'))


nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                440       
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 671
Trainable params: 671
Non-trainable params: 0
_________________________________________________________________


In [27]:

nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 740us/step - loss: 0.5872 - accuracy: 0.7029
Epoch 2/100
804/804 [==============================] - 1s 709us/step - loss: 0.5621 - accuracy: 0.7284
Epoch 3/100
804/804 [==============================] - 1s 795us/step - loss: 0.5577 - accuracy: 0.7320
Epoch 4/100
804/804 [==============================] - 1s 906us/step - loss: 0.5555 - accuracy: 0.7314
Epoch 5/100
804/804 [==============================] - 1s 778us/step - loss: 0.5541 - accuracy: 0.7322
Epoch 6/100
804/804 [==============================] - 1s 837us/step - loss: 0.5532 - accuracy: 0.7318
Epoch 7/100
804/804 [==============================] - 1s 839us/step - loss: 0.5524 - accuracy: 0.7328
Epoch 8/100
804/804 [==============================] - 1s 768us/step - loss: 0.5514 - accuracy: 0.7320
Epoch 9/100
804/804 [==============================] - 1s 844us/step - loss: 0.5511 - accuracy: 0.7328
Epoch 10/100
804/804 [==============================] - 1s 714us/step - l

In [28]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 0s - loss: 0.5506 - accuracy: 0.7286 - 228ms/epoch - 852us/step
Loss: 0.550644040107727, Accuracy: 0.7285980582237244


In [29]:
nn.save('..//Deep-Learning-project/models/optimization1.h5')

In [30]:

nn = tf.keras.models.Sequential()

nn_input_dim = X_train.shape[1]

nn_units = nn_input_dim*2

nn.add(tf.keras.layers.Dense(units=nn_units, activation='relu', input_dim=nn_input_dim))

nn.add(tf.keras.layers.Dense(units=nn_units/2, activation='relu'))

nn.add(tf.keras.layers.Dense(units=nn_units/4, activation='tanh'))

nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 86)                3784      
                                                                 
 dense_5 (Dense)             (None, 43)                3741      
                                                                 
 dense_6 (Dense)             (None, 21)                924       
                                                                 
 dense_7 (Dense)             (None, 1)                 22        
                                                                 
Total params: 8,471
Trainable params: 8,471
Non-trainable params: 0
_________________________________________________________________


In [31]:

nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

fit_model = nn.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
804/804 [==============================] - 1s 816us/step - loss: 0.5683 - accuracy: 0.7208
Epoch 2/25
804/804 [==============================] - 1s 843us/step - loss: 0.5552 - accuracy: 0.7299
Epoch 3/25
804/804 [==============================] - 1s 810us/step - loss: 0.5524 - accuracy: 0.7298
Epoch 4/25
804/804 [==============================] - 1s 799us/step - loss: 0.5506 - accuracy: 0.7327
Epoch 5/25
804/804 [==============================] - 1s 792us/step - loss: 0.5497 - accuracy: 0.7321
Epoch 6/25
804/804 [==============================] - 1s 817us/step - loss: 0.5494 - accuracy: 0.7327
Epoch 7/25
804/804 [==============================] - 1s 824us/step - loss: 0.5479 - accuracy: 0.7335
Epoch 8/25
804/804 [==============================] - 1s 804us/step - loss: 0.5476 - accuracy: 0.7337
Epoch 9/25
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7349
Epoch 10/25
804/804 [==============================] - 1s 989us/step - loss: 0.5463 

In [32]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 0s - loss: 0.5505 - accuracy: 0.7291 - 204ms/epoch - 762us/step
Loss: 0.5505492687225342, Accuracy: 0.7290646433830261


In [33]:
nn.save('..//Deep-Learning-project/models/optimization1.h5')

In [34]:

nn = tf.keras.models.Sequential()

nn_input_dim = X_train.shape[1]

nn_units = nn_input_dim*3


nn.add(tf.keras.layers.Dense(units=nn_units, activation='relu', input_dim=nn_input_dim))

nn.add(tf.keras.layers.Dense(units=nn_units/2, activation='relu'))


nn.add(tf.keras.layers.Dense(units=nn_units/2, activation='tanh'))


nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 129)               5676      
                                                                 
 dense_9 (Dense)             (None, 64)                8320      
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 18,221
Trainable params: 18,221
Non-trainable params: 0
_________________________________________________________________


In [35]:

nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 848us/step - loss: 0.5686 - accuracy: 0.7215
Epoch 2/100
804/804 [==============================] - 1s 906us/step - loss: 0.5553 - accuracy: 0.7309
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5536 - accuracy: 0.7296
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5517 - accuracy: 0.7318
Epoch 5/100
804/804 [==============================] - 1s 984us/step - loss: 0.5507 - accuracy: 0.7323
Epoch 6/100
804/804 [==============================] - 1s 998us/step - loss: 0.5494 - accuracy: 0.7322
Epoch 7/100
804/804 [==============================] - 1s 874us/step - loss: 0.5489 - accuracy: 0.7336
Epoch 8/100
804/804 [==============================] - 1s 960us/step - loss: 0.5475 - accuracy: 0.7338
Epoch 9/100
804/804 [==============================] - 1s 861us/step - loss: 0.5468 - accuracy: 0.7352
Epoch 10/100
804/804 [==============================] - 1s 863us/step - loss:

In [36]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 0s - loss: 0.5576 - accuracy: 0.7270 - 207ms/epoch - 772us/step
Loss: 0.557603657245636, Accuracy: 0.7269652485847473


In [37]:
nn.save('..//Deep-Learning-project/models/optimization1.h5')

In [38]:

def create_model(hp):
    nn_model = tf.keras.models.Sequential()

   
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=120,
        step=10), activation=activation, input_dim=nn_input_dim))

   
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=50,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

  
    nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return nn_model

In [39]:
tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=20,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [40]:
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

INFO:tensorflow:Oracle triggered exit


In [41]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 111,
 'num_layers': 3,
 'units_0': 46,
 'units_1': 36,
 'units_2': 41,
 'units_3': 1,
 'units_4': 41,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [42]:
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 0s - loss: 0.5505 - accuracy: 0.7317 - 269ms/epoch - 1ms/step
Loss: 0.5505220293998718, Accuracy: 0.7317471504211426


In [43]:
best_model.save('..//Deep-Learning-project/models/optimization1.h5')